# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_map_csv = "../WeatherPy/City_Weather.csv"
weather_map_df = pd.DataFrame(pd.read_csv(weather_map_csv))
weather_map_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Carnarvon,-24.8667,113.6333,AU,1613188378,87.80,74,90,5.75
1,Palekastro,35.2000,26.2500,GR,1613188380,55.00,70,33,1.99
2,Kahului,20.8947,-156.4700,US,1613188380,80.60,54,40,21.85
3,Dikson,73.5069,80.5464,RU,1613188381,-25.98,87,98,11.39
4,Thompson,55.7435,-97.8558,CA,1613188382,-34.60,74,1,1.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Use API gkey to access info about maps 
gmaps.configure(api_key = g_key)

In [4]:
#Identify locations as lat and lng/humidity as weight -- convert to float to fill NaN values
locations = weather_map_df[["Latitude","Longitude"]]
humidity = weather_map_df["Humidity"].astype(float)

#Plot Google Heatmap
figure = gmaps.figure(zoom_level = 1.80, center = (20, 5))

#Set max intensity based on highest humidity in the df
max_intensity = weather_map_df["Humidity"].max()

#create and combine heat layer on top of Google map
heat_map_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_intensity, point_radius=3)

figure.add_layer(heat_map_layer)

figure

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_weather_df = weather_map_df.loc[(weather_map_df["Max Temperature"] <80) & (weather_map_df["Max Temperature"]>70) & (weather_map_df["Wind Speed"] <10) & (weather_map_df["Cloudiness"]) == 0, :]

#Drop NaNs
vacation_weather_df.dropna()

#reset index
vacation_weather_df.reset_index(drop=True, inplace = True)

vacation_weather_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Carnarvon,-24.8667,113.6333,AU,1613188378,87.80,74,90,5.75
1,Palekastro,35.2000,26.2500,GR,1613188380,55.00,70,33,1.99
2,Kahului,20.8947,-156.4700,US,1613188380,80.60,54,40,21.85
3,Dikson,73.5069,80.5464,RU,1613188381,-25.98,87,98,11.39
4,Thompson,55.7435,-97.8558,CA,1613188382,-34.60,74,1,1.66


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_weather_df[["City", "Latitude", "Longitude", "Country", "Date", "Max Temperature", "Humidity", "Cloudiness", "Wind Speed"]].copy()
hotel_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Carnarvon,-24.8667,113.6333,AU,1613188378,87.80,74,90,5.75
1,Palekastro,35.2000,26.2500,GR,1613188380,55.00,70,33,1.99
2,Kahului,20.8947,-156.4700,US,1613188380,80.60,54,40,21.85
3,Dikson,73.5069,80.5464,RU,1613188381,-25.98,87,98,11.39
4,Thompson,55.7435,-97.8558,CA,1613188382,-34.60,74,1,1.66


In [7]:
#Add "Hotel Name" Column
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Carnarvon,-24.8667,113.6333,AU,1613188378,87.80,74,90,5.75,
1,Palekastro,35.2000,26.2500,GR,1613188380,55.00,70,33,1.99,
2,Kahului,20.8947,-156.4700,US,1613188380,80.60,54,40,21.85,
3,Dikson,73.5069,80.5464,RU,1613188381,-25.98,87,98,11.39,
4,Thompson,55.7435,-97.8558,CA,1613188382,-34.60,74,1,1.66,


In [8]:
#set radius for 5000m for nearby hotels
keyword = "hotel"
radius = 5000

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#pass in parameters
params = {"keyword": keyword, "radius": radius, "key": g_key}

In [9]:
#for loop to request city info within 5000m
for index, row in hotel_df[0:10].iterrows():
    
    #add coordinates columns
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    #include into parameters settings
    params["location"] = f"{latitude}, {longitude}"
    
    #Get API request response and results
    response = requests.get(base_url, params = params).json()
    results = response["results"]
    
    #Add try block to prevent code from breaking
    try:
        #add hotel result to df and print it
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
        print(f"{row['City']}: the nearest hotel is {results[0]['name']}.")
    
    except:
        print(f"{row['City']}: hotel information missing, next result...")

print("-------------------------------------------------------")


Carnarvon: the nearest hotel is Coral Coast Tourist Park.
Palekastro: the nearest hotel is Cavo Plako Villas.
Kahului: the nearest hotel is Maui Beach Hotel.
Dikson: hotel information missing, next result...
Thompson: the nearest hotel is Quality Inn & Suites Thompson.
Jamestown: the nearest hotel is Chautauqua Harbor Hotel.
Port Augusta: the nearest hotel is Majestic Oasis Apartments.
Ushuaia: the nearest hotel is Hotel Austral.
Manono: the nearest hotel is Hotel hewa bora.
Bredasdorp: the nearest hotel is Bredasdorp Country Manor.
-------------------------------------------------------


In [10]:
#Dataframe displaying 10 results
hotels_reduced_df = hotel_df[0:10].copy()
hotels_reduced_df

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Carnarvon,-24.8667,113.6333,AU,1613188378,87.80,74,90,5.75,Coral Coast Tourist Park
1,Palekastro,35.2000,26.2500,GR,1613188380,55.00,70,33,1.99,Cavo Plako Villas
2,Kahului,20.8947,-156.4700,US,1613188380,80.60,54,40,21.85,Maui Beach Hotel
3,Dikson,73.5069,80.5464,RU,1613188381,-25.98,87,98,11.39,
4,Thompson,55.7435,-97.8558,CA,1613188382,-34.60,74,1,1.66,Quality Inn & Suites Thompson
5,Jamestown,42.0970,-79.2353,US,1613188307,14.00,73,1,4.61,Chautauqua Harbor Hotel
6,Port Augusta,-32.5000,137.7667,AU,1613188384,80.01,53,0,10.36,Majestic Oasis Apartments
7,Ushuaia,-54.8000,-68.3000,AR,1613188256,57.20,82,75,10.36,Hotel Austral
8,Manono,-7.3000,27.4167,CD,1613188385,69.75,96,100,2.57,Hotel hewa bora
9,Bredasdorp,-34.5322,20.0403,ZA,1613188386,69.80,78,12,5.75,Bredasdorp Country Manor


In [21]:
#Drop any null info and update df
hotels_reduced_df.dropna()
hotels_reduced_df.head()

,City,Latitude,Longitude,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Carnarvon,-24.8667,113.6333,AU,1613188378,87.80,74,90,5.75,Coral Coast Tourist Park
1,Palekastro,35.2000,26.2500,GR,1613188380,55.00,70,33,1.99,Cavo Plako Villas
2,Kahului,20.8947,-156.4700,US,1613188380,80.60,54,40,21.85,Maui Beach Hotel
3,Dikson,73.5069,80.5464,RU,1613188381,-25.98,87,98,11.39,
4,Thompson,55.7435,-97.8558,CA,1613188382,-34.60,74,1,1.66,Quality Inn & Suites Thompson


In [22]:
hotels_reduced_df.isnull().sum()

City               0
Latitude           0
Longitude          0
Country            0
Date               0
Max Temperature    0
Humidity           0
Cloudiness         0
Wind Speed         0
Hotel Name         0
dtype: int64

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
print(locations)

     Latitude  Longitude
0    -24.8667   113.6333
1     35.2000    26.2500
2     20.8947  -156.4700
3     73.5069    80.5464
4     55.7435   -97.8558
..        ...        ...
540   50.1001   -91.9170
541   38.7518    42.4814
542   47.3087  -122.0032
543   58.8394    36.4321
544   51.7435   111.9606

[545 rows x 2 columns]


In [25]:
# Add marker layer ontop of heat map
pin_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

#Plot second heat map just like original map
heatmap_2 = gmaps.figure(zoom_level = 2, center = (20,5))

#Set highest humidity level as max intensity
humidity = weather_map_df["Humidity"].astype(float)
max_intensity = weather_map_df["Humidity"].max()

print(humidity)
#Create and add heat layers to second figure
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = max_intensity, point_radius = 3)

heatmap_2.add_layer(heat_layer)
heatmap_2.add_layer(pin_markers)

# Display figure
heatmap_2

0      74.0
1      70.0
2      54.0
3      87.0
4      74.0
       ... 
570    69.0
571    93.0
572    79.0
573    89.0
574    84.0
Name: Humidity, Length: 575, dtype: float64


ValueError: weights must be of the same length as locations or None